In [1]:
import DBcm
import swim_utils

In [2]:
config = {
    "user": "swimuser",
    "password": "swimuserpasswd",
    "database": "SwimclubDB",
    "host": "localhost",
}

In [3]:
with DBcm.UseDatabase(config) as db:
    SQL = """ show tables """
    db.execute(SQL)
    results = db.fetchall()

print(results)

[('events',), ('swimmers',), ('times',)]


In [4]:
def upload_data(file_name: str) -> None:
    (
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ) = swim_utils.get_swimmers_data(file_name)

    print((
        name,
        age,
        distance,
        stroke,
        the_times,
        converts,
        the_average,
    ))

    with DBcm.UseDatabase(config) as db:
        SQL = """   INSERT INTO events
                    (name, age, location)
                    VALUES
                    ("Paul", 21, "Carlow")
            """

upload_data("SwimData/swimdata/Abi-10-100m-Back.txt")


('Abi', '10', '100m', 'Back', ['1:31.59', '1:26.55', '1:28.75', '1:39.79', '1:32.37'], [9159, 8655, 8875, 9979, 9237], '1:31.81')
